Import pandas

In [1]:
import pandas as pd

Scrape tables from html to dataframe and select the one we are looking for

In [2]:
#scraping

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html(url)
toronto_fsa_df = tables[0]

Check the results of scraping

In [3]:
# see if it worked correctly

print(toronto_fsa_df.head())
print(toronto_fsa_df.shape)

  Postal Code           Borough               Neighborhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront
(180, 3)


Data cleaning and transformation

In [4]:
# transfomation

# drop rows where Borough is Not assigned
toronto_fsa_df.drop(toronto_fsa_df.loc[toronto_fsa_df['Borough']=="Not assigned"].index, inplace=True)
# there are no neigborhoods with Not assigned value so there is no need to fill in the cells with the asociated Borough name
# leave only one row for each Postal Code and combine the associated neigborhoods into one in that row
toronto_fsa_df.groupby(['Postal Code','Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


Check the shape of the clean dataset

In [5]:
toronto_fsa_df.shape

(103, 3)

Write a function that returns longitude and latitude values for a given Postal Code

In [6]:
import pgeocode # import pgeocode. decided to use this since it seems to be reliable and easy to use

def pc2ll(postalcode,country='CA'):
    nomo = pgeocode.Nominatim(country)
    a = nomo.query_postal_code(postalcode)
    return a[['latitude','longitude']]

Add longitude and latitude columns to the dataframe

In [11]:
toronto_fsa_df['Latitude'] = toronto_fsa_df['Postal Code'].apply(lambda x: pc2ll(x)[0])
toronto_fsa_df['Longitude'] = toronto_fsa_df['Postal Code'].apply(lambda x: pc2ll(x)[1])

Check if se got what we expected

In [13]:
toronto_fsa_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.7545,-79.3300
3,M4A,North York,Victoria Village,43.7276,-79.3148
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889


In [14]:
toronto_fsa_df.shape

(103, 5)

Everything seems to be OK